In [609]:
#dependencies
import pandas as pd
import numpy as np
file = "Resources/purchase_data.csv"
purc_df = pd.read_csv(file)

In [610]:
purc_df.head()
# print columns for future use
# purc_df.['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price']

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [611]:
# find unique user nicknames for player number
player_unique = len(purc_df['SN'].unique())
print("Total number of players: " + ' {:,.0f}'.format(player_unique))

# sum all sales prices for gross revenues
pd.options.display.float_format = '{:,.2f}'.format
gross_sales = purc_df['Price'].sum()
print("Gross Sales: "+ '$ {:,.2f}'.format(gross_sales))

# number of purchases will be equal to number of rows in dataframe 
num_purc = purc_df['Purchase ID'].count()
print("Number of Purchases: " +  ' {:,.0f}'.format(num_purc))

# average purchase price
avg_price = purc_df['Price'].mean()
print("Average Price: "+ '$ {:,.2f}'.format(avg_price))

# number of unique values in items column
unique_items = len(purc_df['Item ID'].unique())
print("Number of unique items: " + ' {:,.0f}'.format(unique_items))


Total number of players:  576
Gross Sales: $ 2,379.77
Number of Purchases:  780
Average Price: $ 3.05
Number of unique items:  183


In [612]:
# Purchasing Analysis cont'd: present in dataframe format
Purc_Analysis = pd.DataFrame({"Gross Sales": [gross_sales],
                              "Purchase Count": [num_purc] ,
                              "Unique Item Cnt": [unique_items] ,
                              "Average Price": [avg_price], 
                              "Player Count" :[player_unique]})

# dataframe formatted in traditonal accounting style
Purc_Analysis['Gross Sales']=Purc_Analysis['Gross Sales'].map("$ {:,.0f} ".format)
Purc_Analysis['Purchase Count']=Purc_Analysis['Purchase Count']
Purc_Analysis['Unique Item Cnt']=Purc_Analysis['Unique Item Cnt']
Purc_Analysis['Average Price']=Purc_Analysis['Average Price'].map("$ {:,.2f} ".format)
Purc_Analysis['Player Count']=Purc_Analysis['Player Count']
Purc_Analysis.head()

,Gross Sales,Purchase Count,Unique Item Cnt,Average Price,Player Count
0,"$ 2,380",780,183,$ 3.05,576


In [613]:
player_unq=purc_df.drop_duplicates(['SN'], keep = 'last')
player_unq.head()
# an overview for gender demographics
gender_df=player_unq['Gender'].value_counts().reset_index()
total = gender_df['Gender'].sum()
# a new column for percentage of total players
gender_df['Perc_Gender (%)'] = gender_df['Gender']/total*100
# gender_df= gender_df.reset_index()
# gender_df=gender_df.drop(columns='level_0')
# * Percentage and Count of Male Players
gender_df.loc[gender_df['index']=='Male','Perc_Gender (%)']
gender_df.loc[gender_df['index']=='Male','Gender']
# Percentage and Count of Female Players
gender_df.loc[gender_df['index']=='Female','Perc_Gender (%)']
gender_df.loc[gender_df['index']=='Female','Gender']
# Percentage and Count of Other / Non-Disclosed
gender_df.loc[gender_df['index']=='Other / Non-Disclosed','Perc_Gender (%)']
int(gender_df.loc[gender_df['index']=='Other / Non-Disclosed','Gender'])
gender_df=gender_df.rename(columns={'index':'Gender','Gender':'Player_Count','Perc_Gender (%)':'Percentage'})
# .map("${:.2f}".format
# gender_df['Percentage']
gender_df['Percentage'] =gender_df['Percentage'].map("{:.2f} %".format)
gender_df.set_index('Gender',inplace=True)
gender_df.rename(columns={'Player_Count':'Player Count'},inplace=True)
gender_df

,Player Count,Percentage
Gender,,
Male,484,84.03 %
Female,81,14.06 %
Other / Non-Disclosed,11,1.91 %


In [618]:
### Purchasing Analysis (Gender)
## The below each broken by gender
# Purchase Count
Purchase_Count=pd.DataFrame(purc_df.groupby('Gender')['Gender'].count())
Purchase_Count.head()

# Avg Purchase Price 
Purchase_Price=pd.DataFrame(purc_df.groupby('Gender')['Price'].mean())
Purchase_Price['Price']=Purchase_Price['Price'].map("{:,.2f}".format)
Purchase_Price

# Total Purchase Value
Total_Purchase = pd.DataFrame(purc_df.groupby('Gender')['Price'].sum())
# Total_Purchase['Price']=Total_Purchase['Price'].map("$  {:,.2f}".format)
Total_Purchase
# Removing duplicates required for "per person" figures
# Avg Purchase Total per Person by Gender
Avg_Purchase=pd.DataFrame(purc_df.groupby('Gender')['Price'].mean())
# Avg_Purchase['Price']=Avg_Purchase['Price'].map("$  {:,.2f}".format)
# Avg_Purchase

Purc_Analysis_Gender =pd.merge(Purchase_Count,Purchase_Price,left_index = True, right_index = True)
Purc_Analysis_Gender_2=pd.merge(Purc_Analysis_Gender,Total_Purchase, left_index = True, right_index = True, suffixes=('_Avg','_Total'))
Purc_Analysis_Gender_2

# Normalize data by deleting all duplicates adn resortt if
remove_dup_df = purc_df.drop_duplicates(subset='SN', keep="last")
norm_df =  pd.DataFrame(remove_dup_df.groupby('Gender')['SN'].count())
Purc_Analysis_Gender_3 = pd.merge(Purc_Analysis_Gender_2, norm_df, left_index = True, right_index = True)
Purc_Analysis_Gender_3['SN']=Purc_Analysis_Gender_3['SN'].map("{:,.0f}".format)
Purc_Analysis_Gender_3

# Normalize data by deleting all duplicates adn resort
# remove_dup_df = purc_df.drop_duplicates(subset='SN', keep="last")
# norm_df =  pd.DataFrame(remove_dup_df.groupby('Gender')['SN'].count())
# Purc_Analysis_Gender_3 = pd.merge(Purc_Analysis_Gender_2, norm_df, left_index = True, right_index = True)
# Purc_Analysis_Gender_3['SN']=Purc_Analysis_Gender_3['SN'].map("{:,.0f}".format)
# Purc_Analysis_Gender_3['Price_Total']=Purc_Analysis_Gender_3['Price_Total'].map("$ {:,.0f}".format)
# Purc_Analysis_Gender_3['Avg_Total']=Purc_Analysis_Gender_3['Price_Total'].sum()/Purc_Analysis_Gender_3['SN']
# Purc_Analysis_Gender_3['Price_Total'].sum()
# Purc_Analysis_Gender_3['NewColumn']=float(Purc_Analysis_Gender_3['Price_Total'])/float(Purc_Analysis_Gender_3['SN'])
# Purc_Analysis_Gender_3['SN'].drop(columns='Gender')
#Purc_Analysis_Gender_3['NewColumn']=float(Purc_Analysis_Gender_2.loc['Male','Price_Total'])/float(norm_df.loc['Male'])
Purc_Analysis_Gender_3
Purc_Analysis_Gender_3['new_column'] = Purc_Analysis_Gender_3['Price_Total'].astype('float')
Purc_Analysis_Gender_3['new_column_2'] = Purc_Analysis_Gender_3['SN'].astype('float')
Purc_Analysis_Gender_3['per person']  = Purc_Analysis_Gender_3['new_column']/Purc_Analysis_Gender_3['new_column_2']
Purc_Analysis_Gender_3

,Gender,Price_Avg,Price_Total,SN
Gender,,,,
Female,113,3.20,361.94,81
Male,652,3.02,"1,967.64",484
Other / Non-Disclosed,15,3.35,50.19,11


In [627]:
Purc_Analysis_Gender_3['new_column'] = Purc_Analysis_Gender_3['Price_Total'].astype('float')
Purc_Analysis_Gender_3['new_column_2'] = Purc_Analysis_Gender_3['SN'].astype('float')
Purc_Analysis_Gender_3['per person']  = Purc_Analysis_Gender_3['new_column']/Purc_Analysis_Gender_3['new_column_2']
Purc_Analysis_Gender_3

,Gender,Price_Avg,Price_Total,SN,new_column,new_column_2,per person
Gender,,,,,,,
Female,113,3.20,361.94,81,361.94,81.00,4.47
Male,652,3.02,"1,967.64",484,"1,967.64",484.00,4.07
Other / Non-Disclosed,15,3.35,50.19,11,50.19,11.00,4.56
